# 非同期推論、同時推論を使った高スループットアプリケーションの基礎を学ぶ
このプログラムでは非同期推論(asynchronous inferencing)を使い、かつ同時に複数の推論要求を実行することにより高い推論スループットを出すプログラムの基礎を学びます。  
プログラムは引き続き、構成の簡単な画像分類(Classification)を題材としています。

### 使用しているシステムの構成を調べる
最初に、ハードウエア構成を調べます。  
`psutil`モジュールをインストールし、CPUのコア数を調べます。

In [ ]:
# Linux
!pip3 install psutil
import psutil
print('# of CPU cores = {}C/{}T'.format(psutil.cpu_count(logical=False), psutil.cpu_count()))

In [ ]:
# Windows
!pip install psutil
import psutil
print('# of CPU cores = {}C/{}T'.format(psutil.cpu_count(logical=False), psutil.cpu_count(logical=True)))

### 入力画像とラベルデータの準備
まずは推論に使用する入力画像ファイルと、クラスラベルのテキストファイルをOpenVINOのdemoディレクトリからコピーしてきます。

In [ ]:
# Linux
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/car.png .
!cp $INTEL_OPENVINO_DIR/deployment_tools/demo/squeezenet1.1.labels synset_words.txt

In [ ]:
# Windows
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\car.png" .
!copy "%INTEL_OPENVINO_DIR%\deployment_tools\demo\squeezenet1.1.labels" synset_words.txt

コピーしてきた推論入力の絵を表示して確認します。

In [ ]:
from IPython.display import Image
Image('car.png')

### 推論に使用するIRモデルデータの準備
推論に使用するモデルを`Model downloader`でダウンロードし、`Model converter`でIRモデルに変換します。

In [ ]:
# Linux
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name googlenet-v3
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name googlenet-v3 --precisions FP16
!ls public/googlenet-v3/FP16 -l

In [ ]:
# Windows
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\downloader.py" --name googlenet-v3
!python "%INTEL_OPENVINO_DIR%\deployment_tools\tools\model_downloader\converter.py"  --name googlenet-v3 --precisions FP16
!dir public\googlenet-v3\FP16

----
ここからPythonプログラム本体となります。  

### プログラムで使用するモジュールをインポートする

In [ ]:
import time

import cv2
import numpy as np
from openvino.inference_engine import IECore

### クラスラベルテキストデータを読み込む

In [ ]:
label = [ s.replace('\n', '') for s in open('synset_words.txt').readlines() ]

### Inference Engineオブジェクトを生成する
- Inference Engineコアオブジェクトを生成
- モデルデータの読み込み
- 入出力ブロブ(バッファ)の情報取得

In [ ]:
# Inference Engineコアオブジェクトの生成
ie = IECore()

# IRモデルファイルの読み込み
model = './public/googlenet-v3/FP16/googlenet-v3'
net = ie.read_network(model=model+'.xml', weights=model+'.bin')

# 入出力blobの名前の取得、入力blobのシェイプの取得
input_blob_name  = list(net.input_info.keys())[0]
output_blob_name = list(net.outputs.keys())[0]
batch,channel,height,width = net.input_info[input_blob_name].tensor_desc.dims

#### (Optional) `Query` API
Inference engineにはQuery APIがあります。キーを指定することでプラグインの情報を取得することが可能です。

In [ ]:
print(ie.get_metric('CPU', 'RANGE_FOR_ASYNC_INFER_REQUESTS'))
print(ie.get_metric('CPU', 'RANGE_FOR_STREAMS'))

#### Plugin configuration
`set_config()` APIを使うことによってInference engineプラグインの設定を行うことが可能です。  
`CPU_THREAD_NUM`, `CPU_BIND_THREAD`, `CPU_THROUGHPUT_STREAMS`などの値をハードウエアに合わせて適切に設定することでパフォーマンスが上がる場合があります。

In [ ]:
ie.set_config({'CPU_THREADS_NUM'       : '0'  }, 'CPU')  # default = 0
ie.set_config({'CPU_BIND_THREAD'       : 'YES' }, 'CPU')  # default = YES
ie.set_config({'CPU_THROUGHPUT_STREAMS': '1'   }, 'CPU')  # default = 1

### モデルをIE coreオブジェクトにロードする
読み込んだネットワークオブジェクトをInference engineプラグインにセットします。  
この時、`num_requests`で推論要求オブジェクト(`infer_request`オブジェクト)をいくつ生成するか指定します。  
`infer_request`の数だけ同時に推論要求を投げることが可能になります。

In [ ]:
exec_net = ie.load_network(network=net, device_name='CPU', num_requests=4)

### コールバック関数をセットする
作成した`infer_request`ひとつづつ個別にコールバック関数を設定します。コールバック関数にラムダ式(無名関数)を指定することも可能です。  
今回コールバック関数の中では推論結果に対して特に何も処理をしていません。単に推論終了カウントを増やす処理だけをしています。

In [ ]:
total_infer=0

def callback(status_code, output):
    global total_infer
    total_infer  += 1

for req in exec_net.requests:
    req.set_completion_callback(callback, req.output_blobs[output_blob_name].buffer[0])

### 推論入力データを準備する
推論入力画像を読み込み、入力ブロブのシェイプに合わせて変形します。

In [ ]:
img = cv2.imread('car.png')
img = cv2.resize(img, (width,height))
img = img.transpose((2, 0, 1))
img = img.reshape((1, channel, height, width))

### **非同期、同時推論**を実行する  
- 推論実行 (非同期推論で400回実行)
- 推論終了待ち
- パフォーマンスデータおよび推論結果の表示

In [ ]:
# Python APIのバグのワークアラウンド。本来不要。一度全`infer_request`に対してダミーの推論を実行する
for req in exec_net.requests:
    req.async_infer(inputs={input_blob_name: img})

infer_slot = 0
total_infer= 0
max_infer = len(exec_net.requests)

start=time.time()

# 推論本体 400回推論を実行
while total_infer<400:
    req = exec_net.requests[infer_slot]
    status = req.wait(0)
    if status == 0 or status==-11:   # infer_requestのステータスが0(OK), -11(INFER_NOT_STARTED)なら推論要求実行
        res = req.async_infer(inputs={input_blob_name: img})
    infer_slot = (infer_slot+1) % max_infer

# 全推論要求が終了するまで待つ
for req in exec_net.requests:
    while req.wait()!=0: pass

# パフォーマンス表示
total=time.time()-start
print('max_infer={} time={:.4}sec fps={}\n'.format(max_infer, total, total_infer/total))

# 推論結果表示
for i, req in enumerate(exec_net.requests):
    output = req.output_blobs[output_blob_name].buffer[0]
    idx = np.argsort(output)[::-1]
    print('infer_request ', i)
    for i in range(5):
        print(idx[i], output[idx[i]], label[idx[i]-1])

----
## ここまでで効率の良い推論プログラムの作り方について学びました
ここでのポイントは下記のものになります。
- 非同期推論を行う
- デバイスの特性に合った同時推論数分、推論要求を投げてデバイスを飽和させる

今回、`CPU_THREAD_NUM`, `CPU_BIND_THREAD`, `CPU_THROUGHPUT_STREAMS`などをデフォルト値と同じ設定で実行しました。    
しかし、これらのパラメータや`num_requests`を使用するプロセッサの構成に合わせて最適化することでさらにパフォーマンスアップすることが可能です。  
パラメーターを調整し、最高のパフォーマンスが出せる設定を狙ってみてください(**DevCloudで実行している場合、うまく設定すると2倍以上速くなる可能性があります**)

<おわり>

~~~cpp
enum StatusCode : int {
  OK = 0, GENERAL_ERROR = -1, NOT_IMPLEMENTED = -2, NETWORK_NOT_LOADED = -3,
  PARAMETER_MISMATCH = -4, NOT_FOUND = -5, OUT_OF_BOUNDS = -6, UNEXPECTED = -7,
  REQUEST_BUSY = -8, RESULT_NOT_READY = -9, NOT_ALLOCATED = -10, INFER_NOT_STARTED = -11,
  NETWORK_NOT_READ = -12
}
~~~